In [0]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import skimage as sk
from tensorflow.python.framework import ops
import idx2numpy
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [0]:
from keras.utils import to_categorical

  Stored in directory: /root/.cache/pip/wheels/7a/b5/69/3e0757b3086607e95db70661798fdf98a77a0bb79c54e1f320
Successfully built idx2numpy


In [7]:
((train_data, train_labels),
(eval_data, eval_labels)) = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
#Normalizing dataset
train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)

In [0]:
#Reshaping dataset
train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
eval_data = eval_data.reshape(eval_data.shape[0], 28, 28, 1)

In [0]:
#Converting to categorical
train_labels = to_categorical(train_labels, 10)
eval_labels = to_categorical(eval_labels, 10)

In [134]:
print(eval_labels[:10,])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


Using keras sequential model for CNN implementation: 

In [0]:
fmnist_model=tf.keras.Sequential()

In [131]:
tf.__version__

'1.14.0-rc1'

Adding first conv layer:


In [0]:
fmnist_model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(28, 28, 1)))

In [132]:
fmnist_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_15 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 7, 7, 32)         

Adding Maxpool layer:

In [0]:
fmnist_model.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides=2))

batch normalization(Optional/It speeds up training):

In [0]:
fmnist_model.add(tf.keras.layers.BatchNormalization())

Adding dropout to our model for regularisation (optional):

In [0]:
fmnist_model.add(tf.keras.layers.Dropout(0.25))
#dropout parameter is float between 0 and 1. Fraction of the input units to drop.

Adding second conv layer to our model:

In [0]:
fmnist_model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'))

Adding 2nd max pool layer:

In [0]:
fmnist_model.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides=2))

implementing batch normalization on second layer(Optional/It speeds up training):

In [0]:
fmnist_model.add(tf.keras.layers.BatchNormalization())

adding dropout to our model for regularization(optional):

In [0]:
fmnist_model.add(tf.keras.layers.Dropout(0.25))
#dropout parameter is float between 0 and 1. Fraction of the input units to drop.

adding third conv layer with batch normalization and dropout:

In [0]:
fmnist_model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'))
fmnist_model.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides=2))
fmnist_model.add(tf.keras.layers.BatchNormalization())
fmnist_model.add(tf.keras.layers.Dropout(0.25))

Flatten and feed it as input to fully connected layer. Also adding a dropout and relu. Softmax for outer layer for 10 class classification:

In [0]:
fmnist_model.add(tf.keras.layers.Flatten())
fmnist_model.add(tf.keras.layers.Dense(288, activation='relu'))
fmnist_model.add(tf.keras.layers.Dropout(0.25))
fmnist_model.add(tf.keras.layers.Dense(10, activation='softmax'))

Fashion MNIST Model Summary:

In [95]:
fmnist_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_15 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 7, 7, 32)         

Compiling the the above three conv and one fully connected layer:

In [0]:
fmnist_model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Training the above model with hyperparameter batch size of 32 and number of epochs= 10

In [110]:
fmnist_model.fit(train_data,
         train_labels,
         batch_size=32,
         epochs=10,
         verbose=1,
         validation_data=(eval_data, eval_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s 169us/sample - loss: 0.2639 - acc: 0.9017 - val_loss: 18.2009 - val_acc: 0.1023
Epoch 2/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2599 - acc: 0.9028 - val_loss: 0.9959 - val_acc: 0.7093
Epoch 3/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2595 - acc: 0.9035 - val_loss: 48.3545 - val_acc: 0.1441
Epoch 4/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2567 - acc: 0.9046 - val_loss: 2.9675 - val_acc: 0.4409
Epoch 5/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2543 - acc: 0.9043 - val_loss: 3.2602 - val_acc: 0.3210
Epoch 6/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2536 - acc: 0.9044 - val_loss: 2.7431 - val_acc: 0.3531
Epoch 7/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2513 - acc: 0.906

Evaluating the model on test data:

In [111]:

score = fmnist_model.evaluate(eval_data, eval_labels, verbose=1)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])

10000/10000 [==============================] - 1s 80us/sample - loss: 0.2434 - acc: 0.9115

 Test accuracy: 0.9115


In [135]:
#Early stopping using keras callback
from tensorflow.keras.callbacks import EarlyStopping

earlyStop=EarlyStopping(monitor="val_acc",verbose=1,mode='min',patience=3)
fmnist_model.fit(train_data,
         train_labels,
         batch_size=32,
         epochs=10,
         validation_data=(eval_data, eval_labels),
         callbacks=[earlyStop])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s 171us/sample - loss: 0.2347 - acc: 0.9119 - val_loss: 0.5810 - val_acc: 0.8097
Epoch 2/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.2334 - acc: 0.9105 - val_loss: 1.8957 - val_acc: 0.4261
Epoch 3/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.2325 - acc: 0.9128 - val_loss: 4.7252 - val_acc: 0.4949
Epoch 4/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.2331 - acc: 0.9128 - val_loss: 1.9326 - val_acc: 0.4953
Epoch 5/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2325 - acc: 0.9127 - val_loss: 4.2564 - val_acc: 0.2371
Epoch 6/10
60000/60000 [==============================] - 10s 165us/sample - loss: 0.2318 - acc: 0.9125 - val_loss: 11.9247 - val_acc: 0.1098
Epoch 7/10
60000/60000 [==============================] - 10s 165us/sample - loss: 0.2308 - acc: 0.9131

In [136]:
score1=fmnist_model.evaluate(eval_data, eval_labels, verbose=1)

10000/10000 [==============================] - 1s 80us/sample - loss: 11.3084 - acc: 0.4114


In [137]:
#print loss and accuracy on test data
print('Test data loss:', score1[0])
print('Test data accuracy:', score1[1])

Test data loss: 11.308396784973144
Test data accuracy: 0.4114


In [0]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [128]:
#ReduceLRonPlateau using keras callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, epsilon=1e-4, mode='min')
fmnist_model.fit(train_data,
         train_labels,
         batch_size=32,
         epochs=10,
         validation_data=(eval_data, eval_labels),
         callbacks=[reduce_lr])

W0618 11:26:44.172710 140642790233984 callbacks.py:1791] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s 170us/sample - loss: 0.2411 - acc: 0.9107 - val_loss: 5.8269 - val_acc: 0.1454
Epoch 2/10
60000/60000 [==============================] - 10s 165us/sample - loss: 0.2403 - acc: 0.9105 - val_loss: 1.9232 - val_acc: 0.6359
Epoch 3/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2414 - acc: 0.9099 - val_loss: 4.7686 - val_acc: 0.4708
Epoch 4/10
60000/60000 [==============================] - 10s 169us/sample - loss: 0.2366 - acc: 0.9116 - val_loss: 6.0719 - val_acc: 0.2498
Epoch 5/10
60000/60000 [==============================] - 10s 165us/sample - loss: 0.2377 - acc: 0.9104 - val_loss: 5.4876 - val_acc: 0.2937
Epoch 6/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.2384 - acc: 0.9100 - val_loss: 6.7707 - val_acc: 0.3633
Epoch 7/10
60000/60000 [==============================] - 10s 167us/sample - loss: 0.2365 - acc: 0.9116 

In [129]:
score2 = fmnist_model.evaluate(eval_data, eval_labels, verbose=1)

10000/10000 [==============================] - 1s 75us/sample - loss: 0.8130 - acc: 0.7394


In [130]:
#print loss and accuracy on test data
print('Test data loss:', score2[0])
print('Test data accuracy:', score2[1])

Test data loss: 0.8130333958625794
Test data accuracy: 0.7394


We can observe that on early stopping and reduce